In [1]:
# List all device
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [2]:
# Check available GPU
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


[]

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";  

In [4]:
# Importing the libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Reshape, Lambda
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.activations import softmax
from keras.optimizers import SGD
from keras.models import load_model
import math
import pickle


In [5]:
with open("../data/dowjones_calculated/periods.txt", "rb") as fp:   # Unpickling
    dataset = pickle.load(fp)

ValueError: unsupported pickle protocol: 3

In [ ]:
X_train = dataset[0][0][0].values
y_train = dataset[0][0][1].values
X_test = dataset[1][0][0].values
y_test = dataset[1][0][1].values

In [ ]:
len(dataset[0])

In [ ]:
def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])
y_train = get_one_hot(y_train, 2)
y_test = get_one_hot(y_test, 2)

In [ ]:
print(f"x train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")
# print(f"predicted shape: {predicted.shape}")

In [ ]:
data = X_train
targets = y_train

train_gen = TimeseriesGenerator(data, targets,
                               length=240, sampling_rate=1,
                               batch_size=510)

In [ ]:
data = X_test
targets = y_test

test_gen = TimeseriesGenerator(data, targets,
                               length=240, sampling_rate=1,
                               batch_size=250)

In [ ]:
X_train = train_gen[0][0]
y_train = train_gen[0][1]
X_test = test_gen[0][0]
y_test = test_gen[0][1]

In [ ]:
print(f"x train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")

In [ ]:
# Reshaping X_train for efficient modelling
# X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
# X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2], 1))

In [ ]:
print(f"x train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")

In [ ]:
for i in range(len(dataset[0])):
    model_period = f"../model/LSTM/my_model1_period{i}.h5"
    regressor = load_model(model_name, custom_objects={"softmax": softmax})
    X_test = dataset[1][i][0].values
    y_test = dataset[1][i][1].values
    y_test = get_one_hot(y_test, 2)
    test_gen = TimeseriesGenerator(X_test, y_test,
                                   length=240, sampling_rate=1,
                                   batch_size=250)

    X_train = train_gen[0][0]
    y_train = train_gen[0][1]
    X_test = test_gen[0][0]
    y_test = test_gen[0][1]
    predicted = regressor.predict(X_test)
    label = predicted > 0.5
    label = label * 1 # Convert boolean to int
    print((sum(y_test[:, :, 1] == label[:, :, 1])/(y_test.size/2)).sum())